# Examen ETL: SPARK y NBA dataset

# MANUEL DEL PINO GUERRERO - MASTER DATA SCIENCE - E.T.L.

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.csv.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

Persist: Persist y caché son muy parecidos. Se trata en concreto de una transformación. Estas funciones son interesantes cuando vas a usar la tabla resultante del persist varias veces, ya que si bien es más lento de ejecutar, una vez se ha ejecutado ya la tienes para posteriores usos y te ahorrarías repetir pasos. La diferencia entre persist y caché está en que caché lo guarda en la RAM y será más rápido. Persist puedes elegir si ram o disco duro.

Collect: En este caso Collect es una acción, por lo que será más rápido de ejecutarse pero al no crear un nuevo objeto, se tendrá que volver a ejecutarse cada vez, por lo que a la larga el proceso será más lento.

Por lo tanto vamos a usar "Persist/Caché" para situaciones en las que vayamos a utilizar los resultados varias veces y "Collect" para casos 'puntuales'.

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

1) Dependiendo del tipo de datos o como se nos presenta la fuente del dataset, tendremos que definir la función a emplear (ya sea en formato csv o desde una url por ejemplo)

2) Establecer la correlación con la fuente.

3) Extraer el dato (txt, csv...).

4) Cargar en Spark, Definir el Spark Context y cargar en Sc.

5) Comprobar lo que se ha cargado haciendo un take(acción) y muestrear.

6) Analizar el formato de los datos. Los delimitadores que tiene etc.

## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

Por ejemplo diría que la colecta de los datos no siga siempre el mismo formato, o que no siempre incluya las mismas variables.
La falta de datos en ciertas variables de cada observación implica un valor NA. Si esa columna fuera fundamental para nuestro trabajo podría invalidar toda el análisis exploratorio del mismo. Existen distintas técnicas que pueden hacer que no tengamos que eliminar la observación. Por ejemplo, si el número de observaciones con NAs fuera reducido, podríamos hacer una labor de investigación(si fuera posible) para completar el dato faltante. Por ejemplo, en un dataset de características de vehículos, falta el consumo de 2 observaciones. En este caso podríamos buscar la ficha técnica de estos vehículos y completar los datos de consumo faltantes. En el caso de que fueran numerosas observaciones, podríamos recurrir a técnicas de predicción. Por ejemplo, si el dataset es lo suficientemente grande y tenemos los suficientes datos, podríamos tratar de completar los NA's haciendo predicciones de dichas observaciones en base al resto de variables. O por ejmplo también poodríamos emplear la técnica de los vecinos más cercanos.

## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

In [1]:
import re
import datetime
import os
import pandas as pd
import datetime as datetime

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row

Vamos a iniciar el SparkContext

In [3]:
sc=SparkContext('local','Simple App')
sqlContext = SQLContext(sc)

·Cargamos el fichero original con la función textFile de Spark Context
·Mapeamos (función map) indicando que el separador es ':'
·Creamos header y le decimos que sea la primera línea (first) (encabezados)

Trás una observación general y previa de los datos en su formato csv, he podido comprobar que existen algunas líneas de texto, concretamente que hace referencia al comienzo de los Playoffs y que por tanto van a hacer que el dataset no sea estable y compatible en formato fecha desde este punto por lo que habrá que realizar una transformación en el mismo para continuar con los ejercicios.

In [4]:
raw_data = sc.textFile("partidosLigaNBA.csv")
data_clean = raw_data.map(lambda x: x.split(":")).filter(lambda x:'Playoffs' not in x[0]).map(lambda x: x)
header = data_clean.first()
csv_data_1 = data_clean.filter(lambda x: x != header)

Le damos formato a cada una de las columnas. Las más 'especiales' en este dataset son la de Date y Start_ET:

In [5]:
DatosNBA = csv_data_1.map(lambda x: Row(Date = datetime.datetime.strptime((x[0]),'%a, %b %d, %Y').strftime('%m/%d/%Y'),
                                               Start_ET= (x[1]+' : '+x[2]),
                                               Visitor= x[3],
                                               Pts_Visitor = int(x[4]), 
                                               Home = x[5],
                                               Pts_Local = int(x[6])))

"Resgistramos" interactions, que será el nombre de la tabla SQL

In [6]:
interactions_df = sqlContext.createDataFrame(DatosNBA)
interactions_df.registerTempTable("interactions")

Comprobamos que el dataset se ha cargado correctamente con .show

In [8]:
interactions_df.show()

+----------+--------------------+---------+-----------+-----------+--------------------+
|      Date|                Home|Pts_Local|Pts_Visitor|   Start_ET|             Visitor|
+----------+--------------------+---------+-----------+-----------+--------------------+
|10/30/2007|Golden State Warr...|       96|        117|"7 : 30 pm"|           Utah Jazz|
|10/30/2007|  Los Angeles Lakers|       93|         95|"7 : 30 pm"|     Houston Rockets|
|10/30/2007|   San Antonio Spurs|      106|         97|"7 : 00 pm"|Portland Trail Bl...|
|10/31/2007| Cleveland Cavaliers|       74|         92|"8 : 00 pm"|    Dallas Mavericks|
|10/31/2007|      Denver Nuggets|      120|        103|"8 : 30 pm"| Seattle SuperSonics|
|10/31/2007|      Indiana Pacers|      119|        110|"7 : 00 pm"|  Washington Wizards|
|10/31/2007|   Memphis Grizzlies|      101|        104|"7 : 00 pm"|   San Antonio Spurs|
|10/31/2007|     New Jersey Nets|      112|        103|"7 : 30 pm"|       Chicago Bulls|
|10/31/2007| New Orle

## Quinta tarea: Calcular la media de la diferencia de puntos por año

En primer lugar, creamos una columna con la diferencia de puntos por partido. Acto seguido creamos otra columna que será el valor absoluto de la anterior.

In [9]:
from  pyspark.sql.functions import abs

interactions_df=interactions_df.withColumn('dif', interactions_df.Pts_Local - interactions_df.Pts_Visitor)
interactions_df = interactions_df.withColumn('dif_abs',abs(interactions_df.dif))
interactions_df.show()

+----------+--------------------+---------+-----------+-----------+--------------------+---+-------+
|      Date|                Home|Pts_Local|Pts_Visitor|   Start_ET|             Visitor|dif|dif_abs|
+----------+--------------------+---------+-----------+-----------+--------------------+---+-------+
|10/30/2007|Golden State Warr...|       96|        117|"7 : 30 pm"|           Utah Jazz|-21|     21|
|10/30/2007|  Los Angeles Lakers|       93|         95|"7 : 30 pm"|     Houston Rockets| -2|      2|
|10/30/2007|   San Antonio Spurs|      106|         97|"7 : 00 pm"|Portland Trail Bl...|  9|      9|
|10/31/2007| Cleveland Cavaliers|       74|         92|"8 : 00 pm"|    Dallas Mavericks|-18|     18|
|10/31/2007|      Denver Nuggets|      120|        103|"8 : 30 pm"| Seattle SuperSonics| 17|     17|
|10/31/2007|      Indiana Pacers|      119|        110|"7 : 00 pm"|  Washington Wizards|  9|      9|
|10/31/2007|   Memphis Grizzlies|      101|        104|"7 : 00 pm"|   San Antonio Spurs| -3

A continuación importamos sql.functions para ejecutar .split y crear una columna que sea el año a partir del campo fecha.

In [10]:
import pyspark.sql.functions as psf
split_date = psf.split(interactions_df['Date'], '/')     
interactions_df= interactions_df.withColumn('Year', split_date.getItem(2))

In [11]:
interactions_df.show()

+----------+--------------------+---------+-----------+-----------+--------------------+---+-------+----+
|      Date|                Home|Pts_Local|Pts_Visitor|   Start_ET|             Visitor|dif|dif_abs|Year|
+----------+--------------------+---------+-----------+-----------+--------------------+---+-------+----+
|10/30/2007|Golden State Warr...|       96|        117|"7 : 30 pm"|           Utah Jazz|-21|     21|2007|
|10/30/2007|  Los Angeles Lakers|       93|         95|"7 : 30 pm"|     Houston Rockets| -2|      2|2007|
|10/30/2007|   San Antonio Spurs|      106|         97|"7 : 00 pm"|Portland Trail Bl...|  9|      9|2007|
|10/31/2007| Cleveland Cavaliers|       74|         92|"8 : 00 pm"|    Dallas Mavericks|-18|     18|2007|
|10/31/2007|      Denver Nuggets|      120|        103|"8 : 30 pm"| Seattle SuperSonics| 17|     17|2007|
|10/31/2007|      Indiana Pacers|      119|        110|"7 : 00 pm"|  Washington Wizards|  9|      9|2007|
|10/31/2007|   Memphis Grizzlies|      101|   

Y calculamos la diferencia media por partido cadapor año:

In [12]:
media1 = interactions_df.groupBy('Year').avg('dif_abs')
media1.show()

+----+------------------+
|Year|      avg(dif_abs)|
+----+------------------+
|2016|11.550637659414853|
|2012|10.845318860244234|
|2017|11.422166874221668|
|2014|  10.9047976011994|
|2013|11.071752265861027|
|2009|11.090425531914894|
|2011|10.661016949152541|
|2008|11.543543543543544|
|2007|11.096491228070175|
|2015|11.159969673995452|
|2010| 10.86903860711582|
+----+------------------+



## Sexta tarea: ¿Han judado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

Creamos una nueva tabla ejecutando una query que nos saque el count de veces que aparece cada equipo como local:

In [13]:
query2 = "SELECT Home, COUNT(Home) as Partidos_Local FROM interactions GROUP BY Home"
NPartidosLoc = sqlContext.sql(query2)
NPartidosLoc.show()

+--------------------+--------------+
|                Home|Partidos_Local|
+--------------------+--------------+
|        Phoenix Suns|           412|
|      Boston Celtics|           467|
|    Dallas Mavericks|           431|
|New Orleans Pelicans|           166|
|       Brooklyn Nets|           217|
|     New York Knicks|           412|
| New Orleans Hornets|           250|
|   Memphis Grizzlies|           433|
|Minnesota Timberw...|           402|
|  Los Angeles Lakers|           450|
|Golden State Warr...|           445|
|       Orlando Magic|           431|
|   Charlotte Bobcats|           283|
|Los Angeles Clippers|           431|
|     Detroit Pistons|           415|
|       Chicago Bulls|           436|
|     Milwaukee Bucks|           413|
| Cleveland Cavaliers|           449|
|      Indiana Pacers|           434|
|  Washington Wizards|           421|
+--------------------+--------------+
only showing top 20 rows



Repetimos el proceso con los Visitor:

In [14]:
queryVisitor = "SELECT Visitor, COUNT(Visitor) as Partidos_Visitor FROM interactions GROUP BY Visitor"
NPartidosVis = sqlContext.sql(queryVisitor)
NPartidosVis.show()

+--------------------+----------------+
|             Visitor|Partidos_Visitor|
+--------------------+----------------+
|        Phoenix Suns|             413|
|      Boston Celtics|             463|
|    Dallas Mavericks|             436|
|New Orleans Pelicans|             166|
|       Brooklyn Nets|             218|
|     New York Knicks|             413|
| New Orleans Hornets|             249|
|Minnesota Timberw...|             402|
|   Memphis Grizzlies|             434|
|  Los Angeles Lakers|             447|
|Golden State Warr...|             440|
|       Orlando Magic|             432|
|   Charlotte Bobcats|             283|
|Los Angeles Clippers|             430|
|     Detroit Pistons|             414|
|       Chicago Bulls|             437|
|     Milwaukee Bucks|             414|
| Cleveland Cavaliers|             452|
|      Indiana Pacers|             434|
|  Washington Wizards|             423|
+--------------------+----------------+
only showing top 20 rows



Podemos comprobar sin necesidad de sumar ambas tablas, que el número de partidos no es el mismo.

Esto puede deberse al tipo eliminatoria (Playoffs por ejemplo), en el que sólo algunos se clasifican para Play-Offs y por lo tanto estos van a jugar más partidos.

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Clevelant?

En esta ocasión emplearemos una línea de código ya utilizada anteriormente para conocer el año, pero esta vez sacaremos la columna Month.

In [15]:
interactions_df1= interactions_df.withColumn('Month', split_date.getItem(0))

In [16]:
interactions_df1.show()

+----------+--------------------+---------+-----------+-----------+--------------------+---+-------+----+-----+
|      Date|                Home|Pts_Local|Pts_Visitor|   Start_ET|             Visitor|dif|dif_abs|Year|Month|
+----------+--------------------+---------+-----------+-----------+--------------------+---+-------+----+-----+
|10/30/2007|Golden State Warr...|       96|        117|"7 : 30 pm"|           Utah Jazz|-21|     21|2007|   10|
|10/30/2007|  Los Angeles Lakers|       93|         95|"7 : 30 pm"|     Houston Rockets| -2|      2|2007|   10|
|10/30/2007|   San Antonio Spurs|      106|         97|"7 : 00 pm"|Portland Trail Bl...|  9|      9|2007|   10|
|10/31/2007| Cleveland Cavaliers|       74|         92|"8 : 00 pm"|    Dallas Mavericks|-18|     18|2007|   10|
|10/31/2007|      Denver Nuggets|      120|        103|"8 : 30 pm"| Seattle SuperSonics| 17|     17|2007|   10|
|10/31/2007|      Indiana Pacers|      119|        110|"7 : 00 pm"|  Washington Wizards|  9|      9|2007

Actualizamos la tabla SQL, ahora pasa a llamarse interactions_df1, que incluye los campos Year y Month

In [17]:
interactions_df1 = interactions_df1.createOrReplaceTempView("interactions_df1")

Creamos dos campos; Victoria local si los puntos local superan a los visitantes. En ese caso el nuevo campo tomara el valor = 1, y hacemos lo mismo para visitantes.

In [18]:
victorias_partido = sqlContext.sql("Select Month, Home, Visitor,IF(Pts_Local > Pts_Visitor,1,0) AS Victoria_local, IF(Pts_Visitor > Pts_Local,1,0) as Victoria_visitante FROM (SELECT * from Interactions_df1) CONSULTA")

In [19]:
victorias_partido.show()

+-----+--------------------+--------------------+--------------+------------------+
|Month|                Home|             Visitor|Victoria_local|Victoria_visitante|
+-----+--------------------+--------------------+--------------+------------------+
|   10|Golden State Warr...|           Utah Jazz|             0|                 1|
|   10|  Los Angeles Lakers|     Houston Rockets|             0|                 1|
|   10|   San Antonio Spurs|Portland Trail Bl...|             1|                 0|
|   10| Cleveland Cavaliers|    Dallas Mavericks|             0|                 1|
|   10|      Denver Nuggets| Seattle SuperSonics|             1|                 0|
|   10|      Indiana Pacers|  Washington Wizards|             1|                 0|
|   10|   Memphis Grizzlies|   San Antonio Spurs|             0|                 1|
|   10|     New Jersey Nets|       Chicago Bulls|             1|                 0|
|   10| New Orleans Hornets|    Sacramento Kings|             1|            

Sacamos una nueva tabla a partir de la anterior con sólo los partidos del mes número 1 el cual es Enero.

In [20]:
victorias_enero = victorias_partido.filter(victorias_partido.Month == 1)

El siguiente paso a realizar es simplemete para comprobar que tiene sentido la tabla

In [21]:
victorias_enero.filter(victorias_enero.Home == 'Cleveland Cavaliers').show()

+-----+-------------------+--------------------+--------------+------------------+
|Month|               Home|             Visitor|Victoria_local|Victoria_visitante|
+-----+-------------------+--------------------+--------------+------------------+
|   01|Cleveland Cavaliers|       Atlanta Hawks|             1|                 0|
|   01|Cleveland Cavaliers|    Sacramento Kings|             1|                 0|
|   01|Cleveland Cavaliers| Seattle SuperSonics|             1|                 0|
|   01|Cleveland Cavaliers|   Charlotte Bobcats|             1|                 0|
|   01|Cleveland Cavaliers|  Washington Wizards|             1|                 0|
|   01|Cleveland Cavaliers|        Phoenix Suns|             0|                 1|
|   01|Cleveland Cavaliers|       Chicago Bulls|             1|                 0|
|   01|Cleveland Cavaliers|   Charlotte Bobcats|             1|                 0|
|   01|Cleveland Cavaliers|      Boston Celtics|             1|                 0|
|   

Y sumarizamos por enero(mes 1) y victoria local de Cleveland y lo mismo con visitantes.

In [22]:
victorias_enero.filter(victorias_enero.Home == 'Cleveland Cavaliers').groupBy().sum('victoria_local').show()

+-------------------+
|sum(victoria_local)|
+-------------------+
|                 42|
+-------------------+



Aquí podemos observar que Clevelans Cavaliers tiene 42 victorias en el mes de Enero como local

In [23]:
victorias_enero.filter(victorias_enero.Visitor == 'Cleveland Cavaliers').groupBy().sum('victoria_visitante').show()

+-----------------------+
|sum(victoria_visitante)|
+-----------------------+
|                     41|
+-----------------------+



y 41 como visitante en Enero

In [24]:
42 + 41

83

Podemos ver que el equipo Cleveland Cavaliers tiene un total de 83 victorias en el mes de Enero

## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa?

Ya tenemos hechas las tablas del ejercicio anterior asique simplemente será cambiar la consulta:

Queremos la suma de victorias de los Warriors como local:

In [25]:
victorias_partido.filter(victorias_partido.Home == 'Golden State Warriors').groupBy().sum('victoria_local').show()

+-------------------+
|sum(victoria_local)|
+-------------------+
|                308|
+-------------------+



In [26]:
victorias_partido.filter(victorias_partido.Visitor == 'Golden State Warriors').groupBy().sum('victoria_visitante').show()

+-----------------------+
|sum(victoria_visitante)|
+-----------------------+
|                    215|
+-----------------------+



Y la suma de victoria de los Warriors como visitante

Por lo tanto y respondiendo a la octava pregunta, podemos ver que los Warriors son mejores como locales con 308 victorias mientras que 215 son fuera.

## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más)

In [27]:
victorias_partido.groupBy('Home').sum('victoria_local').sort('sum(victoria_local)').show()

+--------------------+-------------------+
|                Home|sum(victoria_local)|
+--------------------+-------------------+
| Seattle SuperSonics|                 13|
|   Charlotte Hornets|                 73|
|     New Jersey Nets|                 76|
|New Orleans Pelicans|                 92|
|       Brooklyn Nets|                107|
|   Charlotte Bobcats|                140|
| New Orleans Hornets|                145|
|Minnesota Timberw...|                148|
|    Sacramento Kings|                172|
|  Philadelphia 76ers|                180|
|     New York Knicks|                200|
|     Milwaukee Bucks|                212|
|     Detroit Pistons|                220|
|  Washington Wizards|                220|
|        Phoenix Suns|                233|
|       Orlando Magic|                243|
|     Toronto Raptors|                244|
|Los Angeles Clippers|                261|
|   Memphis Grizzlies|                268|
|           Utah Jazz|                277|
+----------

## Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com

Por un lado (\d+) nos macheará los dígitos, con lo que conseguiremos los telefonos.

Para el email necesitaremos la siguiente estructura(estructura de un mail estandar): texto+@]texto+punto+texto

Por lo tanto la expresión regular final será la siguiente:

(\d+)|(\w+@\w+.\w+)